# Analysing LAPD Output generated through Plasma VLab

You have two options
1. Use the provided sample data path
OR
2. Provide the path to your own experiment data. This path can be obtained with your own executed experiments in Plasma VLab

In [ ]:
%pip install -qU "airavata-python-sdk[notebook]"
import airavata_jupyter_magic

%authenticate

# PLEASE NOTE: At a given time, ONLY run a single analysis in one of the available clusters: Anvil OR Jetstream. 
# Make sure the resoruce you want to execute on is enabled.

# Anvil
#%request_runtime test_cpu --file=cybershuttle.yml --group=Gkeyll --walltime=60 --use=AnvilCPU:shared

# Jetstream
%request_runtime test_cpu --file=cybershuttle.yml --group=Gkeyll --walltime=60 --use=Gkeyll:cloud

%wait_for_runtime test_cpu --live
%switch_runtime test_cpu

In [ ]:
#Provide your own LAPD output path from https://vlab.plasmascience.scigap.org/ experiment. If not, use the provided Example path

#Example Path
data_path = "/export/vlab_workdirs/LDAP-Sample/"


#Your LAPD Path (Comment the above Example data path. Add your Path here and uncomment.)
#data_path = "/export/vlab_workdirs/PROCESS_79949f46-eb69-4425-8cd3-e2cd95b07c37/"

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import postgkyl as pg
import numpy as np
import shutil

SMALL_SIZE = 16
MEDIUM_SIZE = 18
BIGGER_SIZE = 22
plt.rc('font', size=MEDIUM_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=BIGGER_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=MEDIUM_SIZE)    # legend fontsize
plt.rc('figure', titlesize=MEDIUM_SIZE)  # fontsize of the figure title
plt.rcParams["font.weight"] = "bold"
plt.rcParams["axes.labelweight"] = "bold"

#Note: This cell may execute longer if the job is not yet active in the remote cluster. Please pause till the status =READY message appears.

In [ ]:
# Physical constants and derived parameters
gas_gamma = 5.0 / 3.0 # Adiabatic index.
cFac = 1000 # Scales the speed of light
epsilon0 = 8.854e-12*cFac # Permittivity of free space.
mu0 =  1.257e-6 # Permeability of free space.
light_speed = 1.0/np.sqrt(epsilon0*mu0) # Speed of light. 
eV = 1.6e-19 # electron Volt for charge and temperature
mass_proton = 1.67e-27
mass_ion = 4.*mass_proton # Helium mass.
charge_ion = 1.0*eV # Ion charge.
mass_elc = mass_ion / 100 # Reduced electron mass.
charge_elc = -1.0*eV # Electron charge.
Te_over_Ti = 5.0 # Electron temperature / ion temperature.
n0 = 7.e18 # Number density.
wpi = np.sqrt(charge_ion**2 * n0 / (epsilon0 * mass_ion)) # Ion plasma frequency. 
wpe = np.sqrt(charge_ion**2 * n0 / (epsilon0 * mass_elc)) # Electron plasma frequency. 
di = light_speed/wpi # Ion inertial length. 
de = light_speed/wpe # Electron inertial length. 
B0 = 0.08 # Reference magnetic field strength at z = 0.
omega_ci0 = charge_ion * B0 / mass_ion # Reference ion cyclotron frequency. 
vA0p = B0/np.sqrt(mu0*n0*mass_ion) # Reference Alfven speed. 

### $\texttt{Gkeyll}$ and $\texttt{postgkyl}$

The code we will be using is [$\texttt{Gkeyll}$](https://gkeyll.readthedocs.io/), a general purpose simulation framework for a variety of fluid and plasma systems. You can download and install $\texttt{Gkeyll}$ yourself by following the installation instructions on our [Github repo](https://github.com/ammarhakim/gkylzero). 

To read the data, we will utilize the post-processing suite we have developed alongside $\texttt{Gkeyll}$, [$\texttt{postgkyl}$](https://github.com/ammarhakim/postgkyl), which you can also download and install via the instructions on Github. The cluster we will be utilizing for analyzing the results of our simulations already has installations of $\texttt{Gkeyll}$ and $\texttt{postgkyl}$; we already imported postgkyl in this Jupyter Notebook, so if we did not have $\texttt{postgkyl}$, that import command would not have worked! 

The output of $\texttt{Gkeyll}$ simulations can be manipulated in one of two ways: through the GData class, which retains useful metadata from the simulation to subsequent operations, or by directly fetching the raw values and grid and storing them in Numpy arrays for our subsequent manipulations.

In [ ]:
def read_data(filebase, data_type, frame, zs=[None]*6):
    raw_data = pg.data.GData("%s-%s_%d.gkyl" % (filebase, data_type, frame), z0=zs[0], z1=zs[1], z2=zs[2], z3=zs[3], z4=zs[4], z5=zs[5]) 
    data = raw_data.get_values()  
    time = raw_data.ctx['time']
    # Same grid for all electrons, ions, and EM fields
    coords = raw_data.get_grid()
    # Center the grid values
    for d in range(len(coords)):
        coords[d] = 0.5*(coords[d][:-1] + coords[d][1:])

    return coords, data, time

def data_range(coords, lowerLimits, upperLimits):
    dims = len(coords)
    zs = [None]*6
    for d in range(dims):
        idxs = np.searchsorted(coords[d], [lowerLimits[d], upperLimits[d]])
        if idxs[0] == idxs[1]: 
            zs[d] = '{0}'.format(idxs[0])
        else:
            zs[d] = '{0}:{1}'.format(idxs[0], idxs[1])
    return zs

In [ ]:
filebase = data_path + "LAPD3D5Mg0" #Filename base
startFrame = 0; endFrame = 150; skipFrames = 1 #Start and end frames to be read. skipFrames defines the stride
lowerLimits = [0.,  0., 0.2] #lowerLimits of data to read in units defined in input file
upperLimits = [0.,  0.,  1.e6] #upperLimits of data

Here, we will only be looking at EM field data; however, you can read in moment data by changing data_type to elc or ion, where the names can be determined from the output filenames. The final index in the data array is the component of the EM field or moment data. A glossary of the components can be found below

field

0 - 2 = Ex, Ey, Ez

3 - 5 = Bx, By, Bz

6, 7 = auxilliary fields

moment data

0 = rho = mass*density

1 - 3 = rho*u_i = momentum density

4 = energy =  density * temperature / (gas_gamma - 1) + 0.5 * rho (ux^2 + uy^2 + uz^2)

In [ ]:
data_type = "field" #field for EM fields, elc for electron moments, ion for ion moments. Follows naming convention of output files.
coords, data, time = read_data(filebase, data_type, startFrame) #Initial read needed for data range selection
zs = data_range(coords, lowerLimits, upperLimits) #Setup range of data to read

In [ ]:
#Read in all selected EM field data

frames = np.arange(startFrame, endFrame+1, skipFrames); nt=len(frames)
timeseries = []
time = np.zeros(nt)
for it in range(nt):
    coords, data, time[it] = read_data(filebase, data_type, frames[it], zs)
    timeseries.append(data)
timeseries = np.array(timeseries) #Array shape (time, x, y, z, component)

In [ ]:
Ex = timeseries[...,0]
Ey = timeseries[...,1]
Bx = timeseries[...,3]
By = timeseries[...,4]
z = coords[2]

maxLim = np.max(abs(By))
plt.figure()
plt.pcolormesh(time*1e6, z, By[:,0,0,:].transpose(), cmap='bwr', shading='gouraud')
plt.clim(-maxLim, maxLim)
plt.colorbar()
plt.xlabel(r'$t(\mu s)$')
plt.ylabel(r'$z(m)$')
plt.title(r'$B_y(x=0,y=0,z,t)(T)$')
plt.show()


In [ ]:
#Compute and plot axial Poynting flux
Sz = (Ex*By - Ey*Bx) / mu0 
iz = np.searchsorted(z, 3.) #Find index for z=3m

plt.figure()
plt.plot(time*1e6, Sz[:,0,0,iz].transpose(),'k', linewidth=2)
plt.plot(time*1e6, np.zeros(nt), 'k')
plt.xlabel(r'$t(\mu s)$')
plt.ylabel(r'$S_z(W/m^2)$')
plt.title(r'$S_z(x=0,y=0,z=3m,t)$')
plt.autoscale(enable=True, axis='both', tight=True)
plt.show()

In [ ]:
%stop_runtime test_cpu
%switch_runtime local